1. Objetivo de la transformación

El objetivo principal es adaptar los datos brutos del dataset para que resulten más comprensibles para un usuario general, evitando valores numéricos que puedan resultar confusos (como concentraciones químicas o cifras absolutas de empleo). Para ello, se aplican dos grandes transformaciones:

- Variables de calidad del aire → se convierten en niveles cualitativos (p. ej. “Bueno”, “Moderado”, “Alto”, “Muy alto”).

Cada valor numérico se clasifica en una categoría cualitativa, por ejemplo:

- “Bueno”

- “Moderado”

- “Dañino para grupos sensibles”

- “Malo”

- “Muy alto”

Por ejemplo, para el índice AQI:

- 0-50 → Bueno

- 51-100 → Moderado

- 101-150 → Sensible

- 151-200 → Malo

- 201-300 → Muy Malo

- +300 → Muy alto

Este proceso se aplicará a cada contaminante usando umbrales específicos para CO, NO₂, O₃, PM10, PM2.5 y SO₂.



- Variables de empleo (paro y afiliación) → se expresan como porcentajes relativos a la población de cada municipio.

Estas transformaciones facilitan la interpretación y mejoran la utilidad del dataset para visualizaciones y toma de decisiones.

In [1]:
import pandas as pd

# ---------------------------------------------------------
# 1. Cargar CSV de datos limpios
# ---------------------------------------------------------
df = pd.read_csv("../dataset/processed_data/df_clean.csv")

# ---------------------------------------------------------
# 2. Definir rangos para clasificar calidad del aire
# ---------------------------------------------------------

def clasificar_aqi(valor):
    if valor <= 50: return "Bueno"
    elif valor <= 100: return "Moderado"
    elif valor <= 150: return "Sensibles en riesgo"
    elif valor <= 200: return "Malo"
    # elif valor <= 300: return "Muy malo"
    else: return "Muy malo"

def clasificar_pm25(valor):
    if valor <= 10: return "Bueno"
    elif valor <= 20: return "Moderado"
    elif valor <= 25: return "Alto"
    #elif valor <= 50: return "Muy alto"
    else: return "Muy malo"

def clasificar_pm10(valor):
    if valor <= 20: return "Bueno"
    elif valor <= 40: return "Moderado"
    elif valor <= 50: return "Alto"
    #elif valor <= 100: return "Muy alto"
    else: return "Muy alto"

def clasificar_ozono(valor):
    if valor <= 50: return "Bueno"
    elif valor <= 100: return "Moderado"
    elif valor <= 130: return "Alto"
    else: return "Muy alto"

def clasificar_no2(valor):
    if valor <= 40: return "Bueno"
    elif valor <= 100: return "Moderado"
    elif valor <= 200: return "Alto"
    else: return "Muy alto"

def clasificar_so2(valor):
    if valor <= 20: return "Bueno"
    elif valor <= 80: return "Moderado"
    elif valor <= 250: return "Alto"
    else: return "Muy alto"

def clasificar_co(valor):
    if valor <= 4: return "Bueno"
    elif valor <= 10: return "Moderado"
    elif valor <= 30: return "Alto"
    else: return "Muy alto"

# Mapeo de funciones por contaminante
funciones_calidad_aire = {
    "aq_aqi": clasificar_aqi,
    "aq_co_value": clasificar_co,
    "aq_no2_value": clasificar_no2,
    "aq_o3_value": clasificar_ozono,
    "aq_pm10_value": clasificar_pm10,
    "aq_pm25_value": clasificar_pm25,
    "aq_so2_value": clasificar_so2,
}

# ---------------------------------------------------------
# 3. Aplicar clasificación cualitativa a calidad del aire
# ---------------------------------------------------------

for col, func in funciones_calidad_aire.items():
    if col in df.columns:
        df[col] = df[col].apply(func)

# ---------------------------------------------------------
# 4. Calcular porcentajes de empleo
# ---------------------------------------------------------

if "poblacion" in df.columns:

    # Evitar divisiones por cero
    df["poblacion"] = df["poblacion"].replace(0, pd.NA)

    if "afiliados_total" in df.columns:
        df["economia_empleo_total_pct"] = (
            (df["afiliados_total"] / df["poblacion"]) * 100
        ).round(0)   # ← redondeo a entero

    if "paro_total" in df.columns:
        df["economia_empleo_paro_pct"] = (
            (df["paro_total"] / df["poblacion"]) * 100
        ).round(0)   # ← redondeo a entero

# ---------------------------------------------------------
# 5. Definir mapeo de renombrado y estructura final
# ---------------------------------------------------------

cols_ref = ["cod_municipio", "municipio", 'poblacion']

calidad_aire = list(funciones_calidad_aire.keys())

count_columns_mapping = {
    # Sanidad
    "g_medicos_familia_count": "sanidad_medicos_familia",
    "g_farmacias_count": "sanidad_farmacias",

    # Educación
    "g_colegios_institutos_count": "educacion_colegios_institutos",

    # Transporte
    "g_paradas_bus_count": "transporte_conectividad_paradas_bus",
    "g_estaciones_principales_count": "transporte_conectividad_estaciones",

    # Economía y Empleo
    "economia_empleo_total_pct": "economia_empleo_total_pct",
    "economia_empleo_paro_pct": "economia_empleo_paro_pct",

    # Bienestar
    "g_gimnasios_count": "bienestar_gimnasios",
    "g_parques_count": "bienestar_parques",
    **{col: f"bienestar_{col}" for col in calidad_aire},

    # Ocio y cultura
    "g_restaurantes_count": "ocio_cultura_restaurantes",
    "g_cafeterias_count": "ocio_cultura_cafeterias",
    "g_bares_count": "ocio_cultura_bares",
    "g_cines_count": "ocio_cultura_cines",

    # Servicios
    "g_bancos_count": "servicios_bancos",
    "g_cajeros_count": "servicios_cajeros",
    "g_centros_comerciales_count": "servicios_centros_comerciales",
    "g_conveniencia_count": "servicios_conveniencia",
    "g_supermercados_count": "servicios_supermercados",
    "g_gasolineras_count": "servicios_gasolineras",
    "g_bomberos_count": "servicios_bomberos",
    "g_comisarias_count": "servicios_comisarias"
}

# ---------------------------------------------------------
# 6. Crear dataframe final filtrado y renombrado
# ---------------------------------------------------------

cols_to_keep = cols_ref + [col for col in count_columns_mapping if col in df.columns]

df_counts = df[cols_to_keep].copy()
df_counts.rename(columns=count_columns_mapping, inplace=True)

# ---------------------------------------------------------
# 7. Mostrar resultado
# ---------------------------------------------------------
print(df_counts.head())

   cod_municipio          municipio  poblacion  sanidad_medicos_familia  \
0             14       Acebeda (La)       68.0                        0   
1             29            Ajalvir     4946.0                        6   
2             35  Alameda del Valle      256.0                        0   
3             40         Álamo (El)    10413.0                        2   
4             88   Aldea del Fresno     3422.0                        3   

   sanidad_farmacias  educacion_colegios_institutos  \
0                  0                              1   
1                  4                             11   
2                  1                              1   
3                  3                              5   
4                  1                              2   

   transporte_conectividad_paradas_bus  transporte_conectividad_estaciones  \
0                                   20                                   4   
1                                   21                        

In [2]:
# Guardar dataframe en CSV
df_counts.to_csv("../website_data/df_counts_categorias.csv", index=False)